<a href="https://colab.research.google.com/github/Myusuf2/Final_MIS-64061_myusuf2/blob/main/Mukhtar_simple_HP_tune_longformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install -q transformers

In [ ]:
%%capture
!pip install ray[tune]

In [ ]:
%%capture
!pip install rouge_score

In [ ]:
%%capture
!pip install datasets

In [ ]:
# Seed
seed = 42
from tensorflow.random import set_seed
set_seed(seed)

# Utils
import os
import re
import math
import numpy as np
import pandas as pd

# Deep Learning
import tensorflow as tf
import tensorflow.keras as keras
from transformers import LEDTokenizerFast, TFLEDForConditionalGeneration

# Sklearn
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit

# Metrics
!pip install -q rouge_score
!pip install -q datasets
from datasets import load_metric

# Speed Optimization
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

In [ ]:
config = {'paths' : ['/content/drive/MyDrive/debate2019.csv'],
          'pretrained' : 'allenai/led-base-16384',
          'batch_size' : 1,
          'max_lr': 5e-7,
          'epochs' : 1,
          'tok_input' : {'padding' : 'max_length',
                         'truncation' : True,
                         'max_length' : 4096,
                         'add_special_tokens' : True,
                         'return_tensors' : 'tf',
                         'is_split_into_words' : False,
                         'return_offsets_mapping' : False},
          
          'tok_output' : {'padding' : 'max_length',
                          'truncation' : True,
                          'max_length' : 256,
                          'add_special_tokens' : True,
                          'return_tensors' : 'tf',
                          'is_split_into_words' : False,
                          'return_offsets_mapping' : False},
         }

##Dataset

In [ ]:
class Dataset:
    def __init__(self) :
        paths = config['paths']
        self.data = pd.read_csv(paths[0])
        self.data = self.data[self.data['#WordsDocument'] > 1000]
        self.tokenizer = LEDTokenizerFast.from_pretrained(config['pretrained'])
    
    def _split(self):
        # self.data['y_bins'] = pd.cut(self.data['AbsCompressionRatio'], bins = 5)
        # sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.1, random_state = seed)
        # self.data['fold'] = 0
        
        # for t_, v_ in sss.split(self.data.drop('y_bins', axis=1), self.data['y_bins']): 
        #     self.data.loc[t_,'fold'] = 1
        
        self.data['fold'] = 0
        ss = ShuffleSplit(n_splits = 1, test_size = 0.1, random_state = seed)
        for t_, v_ in ss.split(self.data): 
            self.data['fold'].iloc[t_] = 1
        
    def _tokenize(self, i):
        x = self.tokenizer(self.data[self.data['fold'] == i]['Full-Document'].values.tolist(), 
                           **config['tok_input'])  
        
        y = self.tokenizer(self.data[self.data['fold'] == i]['Abstract'].values.tolist(), 
                           **config['tok_output'])
        
        # For text summarization the paper encourage to only set the global attention for the 1st token.
        glob_attn = len(x["input_ids"]) * [[0 for _ in range(len(x["input_ids"][0]))]]
        glob_attn[0][0] = 1
        
        return ({'input_ids': x['input_ids'],
                'attention_mask' : x['attention_mask'],
                'decoder_attention_mask' : y['attention_mask'],
                'global_attention_mask' : tf.constant(glob_attn, dtype = tf.int32)}, y['input_ids'])
    
    def _to_tf(self, ds):
        return tf.data.Dataset.from_tensor_slices(ds).batch(config['batch_size']) \
                                                     .prefetch(1)                                           
    
    def get(self) :
        self._split()
        trainset = self._tokenize(1)
        valset = self._tokenize(0)
        
        return (self._to_tf(trainset),
                self._to_tf(valset))
            

In [ ]:
trainset, valset = Dataset().get()

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


##Model

In [ ]:
keras.backend.clear_session()

class Longformer(keras.Model):
    def __init__(self):
        super(Longformer, self).__init__()
        self.model = TFLEDForConditionalGeneration.from_pretrained(config['pretrained'], 
                                                                   return_dict = True)
        
    def call(self, inputs, training = False):
        x, y = inputs
        outputs = self.model(input_ids = x['input_ids'], 
                             attention_mask = x['attention_mask'], 
                             decoder_attention_mask = x['decoder_attention_mask'],
                             labels = y, 
                             global_attention_mask = x['global_attention_mask'])
        return outputs.loss, outputs.logits
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            loss, logits = self(data, training=True)

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        return {"loss": tf.reduce_mean(loss)}
    
    
    def test_step(self, data):
        loss, logits = self(data, training=False)
        
        return {"loss": tf.reduce_mean(loss)}

In [ ]:
led = Longformer()
led.compile(optimizer = keras.optimizers.Adam(config['max_lr']))
# led.model.from_pretrained('/content/drive/MyDrive/LED-fine-tune')

Downloading:   0%|          | 0.00/648M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFLEDForConditionalGeneration.

Some layers of TFLEDForConditionalGeneration were not initialized from the model checkpoint at allenai/led-base-16384 and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##1Cycle Scheduler

In [ ]:
K = keras.backend

class OneCycleLr(keras.callbacks.Callback):
    def __init__(self,
                 max_lr: float,
                 total_steps: int = None,
                 epochs: int = None,
                 steps_per_epoch: int = None,
                 pct_start: float = 0.3,
                 anneal_strategy: str = "cos",
                 cycle_momentum: bool = True,
                 base_momentum: float = 0.85,
                 max_momentum: float = 0.95,
                 div_factor: float = 25.0,
                 final_div_factor: float = 1e4):

        super(OneCycleLr, self).__init__()

        # validate total steps:
        if total_steps :
            self.total_steps = total_steps
        else:
            self.total_steps = epochs * steps_per_epoch

        self.step_num = 0
        self.step_size_up = float(pct_start * self.total_steps) - 1
        self.step_size_down = float(self.total_steps - self.step_size_up) - 1

        # Validate pct_start
        if anneal_strategy == "cos":
            self.anneal_func = self._annealing_cos
        elif anneal_strategy == "linear":
            self.anneal_func = self._annealing_linear

        # Initialize learning rate variables
        self.initial_lr = max_lr / div_factor
        self.max_lr = max_lr
        self.min_lr = self.initial_lr / final_div_factor

        # Initial momentum variables
        self.cycle_momentum = cycle_momentum
        if self.cycle_momentum:
            self.m_momentum = max_momentum
            self.momentum = max_momentum
            self.b_momentum = base_momentum

        # Initialize variable to learning_rate & momentum
        self.track_lr = []
        self.track_mom = []

    def _annealing_cos(self, start, end, pct):
        cos_out = math.cos(math.pi * pct) + 1
        return end + (start - end) / 2.0 * cos_out

    def _annealing_linear(self, start, end, pct):
        return (end - start) * pct + start

    def set_lr_mom(self):
        if self.step_num <= self.step_size_up:
            # update learining rate
            computed_lr = self.anneal_func(self.initial_lr, self.max_lr, self.step_num / self.step_size_up)
            K.set_value(self.model.optimizer.lr, computed_lr)
            # update momentum if cycle_momentum
            if self.cycle_momentum:
                computed_momentum = self.anneal_func(self.m_momentum, self.b_momentum, self.step_num / self.step_size_up)
                try:
                    K.set_value(self.model.optimizer.momentum,
                                computed_momentum)
                except:
                    K.set_value(self.model.optimizer.beta_1, computed_momentum)
        else:
            down_step_num = self.step_num - self.step_size_up
            # update learning rate
            computed_lr = self.anneal_func(self.max_lr, self.min_lr, down_step_num / self.step_size_down)
            K.set_value(self.model.optimizer.lr, computed_lr)
            # update momentum if cycle_momentum
            if self.cycle_momentum:
                computed_momentum = self.anneal_func(self.b_momentum, self.m_momentum, down_step_num / self.step_size_down)
                try:
                    K.set_value(self.model.optimizer.momentum,
                                computed_momentum)
                except:
                    K.set_value(self.model.optimizer.beta_1, computed_momentum)

    def on_train_begin(self, logs=None):
        # Set initial learning rate & momentum values
        K.set_value(self.model.optimizer.lr, self.initial_lr)
        if self.cycle_momentum:
            try:
                K.set_value(self.model.optimizer.momentum, self.momentum)
            except:
                K.set_value(self.model.optimizer.beta_1, self.momentum)

    def on_train_batch_end(self, batch, logs=None):
        # Grab the current learning rate & momentum
        lr = float(K.get_value(self.model.optimizer.lr))
        try:
            mom = float(K.get_value(self.model.optimizer.momentum))
        except:
            mom = float(K.get_value(self.model.optimizer.beta_1))
        # Append to the list
        self.track_lr.append(lr)
        self.track_mom.append(mom)
        # Update learning rate & momentum
        self.set_lr_mom()
        # increment step_num
        self.step_num += 1

##Training

In [ ]:


scheduler = OneCycleLr(max_lr=config['max_lr'], 
                       steps_per_epoch=trainset.cardinality().numpy(), 
                       epochs=config['epochs'])

checkpoint = keras.callbacks.ModelCheckpoint(filepath = '/content/drive/MyDrive/ckpt_led',
                                             save_best_only = True,
                                             save_weights_only = True)
early = keras.callbacks.EarlyStopping(patience=1, restore_best_weights=True)



In [ ]:
history = led.fit(trainset,
                  validation_data = valset,
                  callbacks = [early, scheduler],
                  verbose = 1,
                  epochs = config['epochs'])

led.model.save_pretrained('/content/drive/MyDrive')

3143/3143 [==============================] - 1173s 355ms/step - loss: 4.2668 - val_loss: 4.1733


In [ ]:
led.summary()

Model: "longformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tfled_for_conditional_gener  multiple                 161894745 
 ation (TFLEDForConditionalG                                     
 eneration)                                                      
                                                                 
Total params: 161,894,745
Trainable params: 161,844,480
Non-trainable params: 50,265
_________________________________________________________________


##Testing

In [ ]:
class TextSummarization():
  def __init__(self, pretrained, tok, beam, temperature):

    self.model = Longformer().model.from_pretrained(pretrained)
    self.tokenizer = LEDTokenizerFast.from_pretrained(tok)
    self.beam = beam
    self.temperature = temperature

  def generate(self, text):
    text = self.tokenizer(text, **config['tok_input']).input_ids
    tokens = self.model.generate(text,
                                 min_length = 0,
                                 max_length = 128,
                                 num_beams = self.beam,
                                 temperature = self.temperature,
                                 do_sample = True,
                                 repetition_penalty = 2.5,
                                 length_penalty = 1,
                                 early_stopping = True).numpy()[0]
    return self.tokenizer.decode(tokens, skip_special_tokens = True)

  def post_processing(self, text):
    text = text[:text.rfind('.')+1]
    return re.sub("[\(\[].*?[\)\]]", "", text)

  def summarize(self, text):
    text = self.generate(text)
    text = self.post_processing(text)
    return text

ts = TextSummarization(pretrained = '/content/drive/MyDrive/Shared_Folder_Colab/LED', tok = 'allenai/led-base-16384',beam = 5, temperature = 1.2)

All model checkpoint layers were used when initializing TFLEDForConditionalGeneration.

Some layers of TFLEDForConditionalGeneration were not initialized from the model checkpoint at allenai/led-base-16384 and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
All model checkpoint layers were used when initializing TFLEDForConditionalGeneration.

Some layers of TFLEDForConditionalGeneration were not initialized from the model checkpoint at /content/drive/MyDrive/Statistical Consulting/LED and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_rouge(id_doc, prediction):
    rouge = load_metric("rouge")
    rouge.add(reference = data['Abstract'].values.tolist()[id_doc], prediction = summary)
    score_abs = rouge.compute()['rougeL']
    rouge = load_metric("rouge")
    rouge.add(reference = data['Extract'].values.tolist()[id_doc], prediction = summary)
    score_ext = rouge.compute()['rougeL']
    return score_abs.mid, score_ext.mid

# def compute_bleu(id_doc, prediction):
#     bleu = load_metric("bleu")
#     score_abs = bleu.compute(predictions=[data['Abstract'].values.tolist()[id_doc].split()], 
#                              references=[[summary.split()]], max_order = 1)['bleu']
#     bleu = load_metric("bleu")
#     score_ext = bleu.compute(predictions=[data['Extract'].values.tolist()[id_doc].split()], 
#                              references=[[summary.split()]], max_order = 1)['bleu']
#     return score_abs, score_ext

In [ ]:


paths = config['paths']
data = pd.read_csv(paths[0])
id_doc = 17330

# print('Abstract :')
# print(data['Abstract'].values.tolist()[id_doc])
# print()
summary = ts.summarize(data['Full-Document'].values.tolist()[id_doc])
# print('Summary :')
# print(summary)



In [ ]:
abs, ext = compute_rouge(id_doc, summary)
print('Score on Abstract :', abs)
print('Score on Extract :', ext)

<ipython-input-37-d978c43a3664>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


Score on Abstract : Score(precision=0.058823529411764705, recall=0.15625, fmeasure=0.08547008547008547)
Score on Extract : Score(precision=0.3176470588235294, recall=0.02708124373119358, fmeasure=0.0499075785582255)


In [ ]:
from ray import tune

# 1. Define an objective function.
def objective(config):
    score = compute_rouge(id_doc, summary)
    return {"score": score}


# 2. Define a search space.
search_space = {
    'tok_input' : {'max_length': tune.choice([4096, 2048, 1024, 512])},
    'tok_output': {'max_length': tune.choice([2048, 1024, 512, 256])}

}

# 3. Start a Tune run and print the best result.
tuner = tune.Tuner(objective, param_space=search_space)
results = tuner.fit()
print(results.get_best_result(metric="score", mode="max").config)


2022-12-07 15:16:07,865	INFO worker.py:1528 -- Started a local Ray instance.
2022-12-07 15:16:09,570	WARNING function_trainable.py:586 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


2022-12-07 15:16:14,890	ERROR ray_trial_executor.py:580 -- Trial objective_14ff4_00000: Unexpected error starting runner.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/ray/tune/execution/ray_trial_executor.py", line 573, in start_trial
    return self._start_trial(trial)
  File "/usr/local/lib/python3.8/dist-packages/ray/tune/execution/ray_trial_executor.py", line 473, in _start_trial
    runner = self._setup_remote_runner(trial)
  File "/usr/local/lib/python3.8/dist-packages/ray/tune/execution/ray_trial_executor.py", line 414, in _setup_remote_runner
    return full_actor_class.remote(**kwargs)
  File "/usr/local/lib/python3.8/dist-packages/ray/actor.py", line 637, in remote
    return actor_cls._remote(args=args, kwargs=kwargs, **updated_options)
  File "/usr/local/lib/python3.8/dist-packages/ray/util/tracing/tracing_helper.py", line 387, in _invocation_actor_class_remote_span
    return method(self, args, kwargs, *_args, **_kwargs)
  File "/usr/lo

None
